<a href="https://colab.research.google.com/github/BrokenShell/DS-Unit-1-Sprint-3-Linear-Algebra/blob/master/module4-clustering/LS_DS_134_Clustering_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# K-Means Clustering

# 1) Use the "Breast Cancer Wisconsin (Diagnostic) Data Set" from Kaggle to try and cluster types of cancer cells. 

Here's the original dataset for your reference:

<https://www.kaggle.com/uciml/breast-cancer-wisconsin-data>

## This is a supervised learning dataset

(Because it has **labels** - The "diagnosis" column.)

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA # You don't necessarily have to use this
from sklearn.cluster import KMeans # You don't necessarily have to use this
from sklearn.preprocessing import StandardScaler # You don't necessarily have to use this

df = pd.read_csv("https://raw.githubusercontent.com/ryanleeallred/datasets/master/Cancer_Cells.csv")
print(df.shape)
df.head()

(569, 33)


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


## Now it's an unsupervised learning dataset

(Because we've removed the diagnosis label) - Use this version.

> Data Cleanup

In [0]:
diagnosis = df['diagnosis'].apply(lambda x: 0 if x == "B" else 1)

In [0]:
diagnosis.head(20)

0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    0
Name: diagnosis, dtype: int64

In [0]:
diagnosis.value_counts()

0    357
1    212
Name: diagnosis, dtype: int64

In [0]:
diagnosis.mean()

0.37258347978910367

In [0]:
df = df.drop('diagnosis', axis=1)
df = df.drop('Unnamed: 32', axis=1)
df = df.drop('id', axis=1)
df.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [0]:
df.shape

(569, 30)

In [0]:
df.isna().sum().any()

False

## Let's do it!

- You might want to do some data exploration to see if you can find specific columns that will help you find distinct clusters of cells
- You might want to use the elbow method to decide on the number of clusters to use.


In [0]:
# Perform K-Means Clustering on the Dataset
kmeans = KMeans(2)
kmeans.fit(df)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

> Kmeans can and will randomize the labels it assigns.

In [0]:
# this is a hack to make sure we get the expected labels each time.
# If I had time I'd come up with a better solution. I still might.
while kmeans.labels_.mean() > 0.5:
    kmeans.fit(df)
kmeans.labels_

array([1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

## Check you work: 

This is something that in a truly unsupervised learning situation **WOULD NOT BE POSSIBLE**. But for educational purposes go back and grab the true diagnosis column (label) from the original dataset. Take your cluster labels and compare them to the original diagnosis column. You can make scatterplots for each to see how they compare or you can calculate a percent accuracy score like: 
\begin{align}
\frac{\text{Num Correct Labels}}{\text{Num Total Observations}}
\end{align}

> I went few steps deeper...

In [0]:
def classify(x, y):
    """
    x is the diagnosis
    y is the prediction
    """
    table = {
        (0,0): 'True Negative',
        (0,1): 'False Positive',
        (1,0): 'False Negative',
        (1,1): 'True Positive',
    }
    return table[(x,y)]

In [0]:
print("Kmeans: Prediction Classification Totals")
kmeans_df = pd.DataFrame()
kmeans_df['diagnosis'] = diagnosis
kmeans_df['prediction'] = kmeans.labels_
kmeans_df['results'] = [classify(d,p) for d, p in kmeans_df.values]
kmeans_df['results'].value_counts()

Kmeans: Prediction Classification Totals


True Negative     356
True Positive     130
False Negative     82
False Positive      1
Name: results, dtype: int64

# 2) Perform PCA on your dataset first and *then* use k-means clustering. 

- You need to standardize your data before PCA.
- First try clustering just on PC1 and PC2 so that you can make a scatterplot of your clustering.
- Then use use a scree plot to decide how many principal components to include in your clustering, and use however many principal components you need in order to retain 90% of the variation of the original dataset



In [0]:
def find_components(data: pd.DataFrame, min_percent=90):
    norm = StandardScaler().fit_transform(data)
    for i in range(1, data.shape[1]):
        pca_n = PCA(i)
        pca_n_data = pca_n.fit_transform(norm)
        result = sum(pca_n.explained_variance_ratio_) * 100
        if result < min_percent:
            continue
        else:
            print(f"PCA {i}: {result:.2f}%")
            return pca_n_data

In [0]:
pca_7_df = pd.DataFrame(find_components(df, 90))
pca_7_df.head()

PCA 7: 91.01%


,0,1,2,3,4,5,6
0,9.192837,1.948583,-1.123166,3.633731,-1.195110,1.411426,2.159429
1,2.387802,-3.768172,-0.529293,1.118264,0.621775,0.028656,0.013334
2,5.733896,-1.075174,-0.551748,0.912082,-0.177086,0.541453,-0.668158
3,7.122953,10.275589,-3.232790,0.152547,-2.960879,3.053421,1.429883
4,3.935302,-1.948072,1.389767,2.940640,0.546747,-1.226495,-0.936234


## Check your work: 

- Compare your PC1, PC2 clustering scatterplot to the clustering scatterplots you made on the raw data
- Calculate accuracy scores for both the PC1,PC2 Principal component clustering and the 90% of explained variance clustering.

How do your accuracy scores -when preprocessing the data with PCA- compare to the accuracy when simply clustering on the raw data?

In [0]:
print("Kmeans: Prediction Classification Totals")
kmeans_df = pd.DataFrame()
kmeans_df['diagnosis'] = diagnosis
kmeans_df['prediction'] = kmeans.labels_
kmeans_df['results'] = [classify(d,p) for d, p in kmeans_df.values]
kmeans_df['results'].value_counts()

Kmeans: Prediction Classification Totals


True Negative     356
True Positive     130
False Negative     82
False Positive      1
Name: results, dtype: int64

In [0]:
print(f"{(356 + 130) / len(diagnosis) * 100:.2f}%")

85.41%


In [0]:
pca_kmeans = KMeans(2)
pca_kmeans.fit(pca_7_df)
while pca_kmeans.labels_.mean() > 0.5:
    pca_kmeans.fit(pca_7_df)
print("PCA 7 + Kmeans: Prediction Classification Totals")
final_df = pd.DataFrame()
final_df['diagnosis'] = diagnosis
final_df['prediction'] = pca_kmeans.labels_
final_df['results'] = [classify(d,p) for d, p in final_df.values]
final_df['results'].value_counts()

PCA 7 + Kmeans: Prediction Classification Totals


True Negative     343
True Positive     175
False Negative     37
False Positive     14
Name: results, dtype: int64

In [0]:
print(f"{(343 + 175) / len(diagnosis) * 100:.2f}%")

91.04


In [0]:
pca_17_df = pd.DataFrame(find_components(df, 99))
pca_17_kmeans = KMeans(2)
pca_17_kmeans.fit(pca_17_df)
while pca_17_kmeans.labels_.mean() > 0.5:
    pca_17_kmeans.fit(pca_17_df)
print("PCA 17 + Kmeans: Prediction Classification Totals")
final_17_df = pd.DataFrame()
final_17_df['diagnosis'] = diagnosis
final_17_df['prediction'] = pca_17_kmeans.labels_
final_17_df['results'] = [classify(d,p) for d, p in final_17_df.values]
final_17_df['results'].value_counts()

PCA 17: 99.11%
PCA 17 + Kmeans: Prediction Classification Totals


True Negative     339
True Positive     176
False Negative     36
False Positive     18
Name: results, dtype: int64

In [0]:
print(f"{(339 + 176) / len(diagnosis) * 100:.2f}%")

90.51%


> Applying PCA before Kmeans made the number of correct predictions go up in general. Great! Interestingly: false negatives went down with PCA however... false positives went up! Is this yet another example of no free lunches? Also - increasing the pca to 17 / 99% made little difference to the final outcome, and actually reduced the over all accuracy by 0.5% - I did not expect a regression here.

# Stretch Goals:

- Study for the Sprint Challenge
- Work on your Data Storytelling Project

> Watching several videos on Linear Algebra to help solidify the content and continue studies.

> I have a fairly clear idea of what I want to work on for my project... D&D Monsters! I already have a website, and I know how to use it.